## 50%数据集

In [1]:
import numpy as np
import pandas as pd
import jieba
import csv
import sklearn

## 载入数据

In [20]:
data = open('Data/50%/train.txt','r',encoding='utf-8')
f=open('Data/50%/50%.csv','a',newline='',encoding='utf-8')

csv_writer = csv.writer(f)

m=data.readlines()

csv_writer.writerow(["title","tag"])

for i in m:
    n=i.strip('\n')
    L=n.split('\t')
    # print(L)
    csv_writer.writerow(L)
    
f.close()
data.close()

In [21]:
data=pd.read_csv('Data/50%/50%.csv')
data.head()

,title,tag
0,生态环境部：一季度全国生态环境质量总体持续改善,0
1,科技战“疫”，全球寻何新“利器”？,0
2,种子方舟：国家农作物种质资源库,0
3,新《地名管理条例》：开启我国地名管理的新时代,0
4,新时代的中国青年： 生逢中华民族发展最好时期,0


## 划分训练集测试集

In [22]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data['title'], data['tag'], random_state=1)



In [23]:
print(x_test.shape[0])
print(type(y_test))

21860
<class 'pandas.core.series.Series'>


## 分词

In [24]:
def fenci(train_data):
    words_df = train_data.apply(lambda x:' '.join(jieba.cut(x)))
    return words_df

x_train_fenci = fenci(x_train)
x_train_fenci[:5]


52110    奔驰 EQ   C 电动 SUV 谍 照曝光   对手 锁定 奥迪 e - tron   Q...
9537         一触即发 ？ 俄罗斯 一 超级 武器 首次 曝光 ， 美国 以后 只有 挨打 的 份儿 ？
8650               超级 赌徒 竟 拿 家人 当 赌注 ， 输掉 赌局 后 妻子 改嫁 儿子 改性
39893                   30 多名 台湾 诈骗犯 亚美尼亚 被控   台媒 : 大陆 没 抢
84206                          上联 ： 遍地 鲜花 摇 落地 ， 如何 对 下联 ？
Name: title, dtype: object

## 停词

In [25]:
infile = open("hit_stopwords.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

## 向量化

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=stopwords, max_features=5000)
vectorizer.fit(x_train_fenci)

D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


CountVectorizer(max_features=5000,
                stop_words=['———', '》），', '）÷（１－', '”，', '）、', '＝（', ':', '→',
                            '℃', '&', '*', '一一', '~~~~', '’', '.', '『', '.一',
                            './', '--', '』', '＝″', '【', '［＊］', '｝＞', '［⑤］］',
                            '［①Ｄ］', 'ｃ］', 'ｎｇ昉', '＊', '//', ...])

In [27]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
#模型训练
classifier.fit(vectorizer.transform(x_train_fenci), y_train)
#使用训练好的模型进行预测
classifier.score(vectorizer.transform(fenci(x_test)), y_test)

0.8222323879231473

## 预测

In [28]:
from sklearn.metrics import classification_report
predict=classifier.predict(vectorizer.transform(fenci(x_test)))
print("每个类别的精确率与召回率\n",classification_report(predict, y_test))

每个类别的精确率与召回率
               precision    recall  f1-score   support

           0       0.66      0.76      0.71      1200
           1       0.76      0.79      0.78      2110
           2       0.82      0.75      0.78      1723
           3       0.81      0.79      0.80      2360
           4       0.86      0.87      0.87      2669
           5       0.88      0.87      0.88      2999
           6       0.87      0.75      0.81      3828
           7       0.88      0.91      0.89      2971
           8       0.73      0.85      0.79      2000

    accuracy                           0.82     21860
   macro avg       0.81      0.82      0.81     21860
weighted avg       0.83      0.82      0.82     21860



In [11]:
f=open('test_5.txt','r',encoding='utf-8')

L=[]
m=f.readlines()
for i in m:
    n=i.strip('\n')
    x=n.split('\t')
    L.append(x[1])

data_to_predict=pd.Series(L)
# data_to_predict=np.matrix(data_to_predict)

In [12]:
print(data_to_predict.shape)

(603,)


In [14]:
predict=classifier.predict(vectorizer.transform(fenci(data_to_predict)))

In [15]:
print(predict)

[5 7 2 5 3 2 5 0 5 7 2 0 4 7 0 4 5 5 7 0 5 2 5 6 4 4 5 4 5 2 0 4 2 8 0 5 5
 5 5 5 5 7 6 5 5 0 4 2 0 1 5 5 5 7 4 5 5 5 2 2 0 5 1 1 5 0 2 6 4 0 6 5 1 0
 5 8 5 5 2 3 5 4 4 5 4 1 5 5 5 5 4 3 5 5 5 5 3 5 3 5 5 5 5 4 4 5 1 3 7 8 3
 2 5 5 3 5 4 5 5 5 0 1 5 7 5 5 1 4 5 5 1 4 4 1 7 6 5 3 0 2 1 5 5 7 0 5 7 1
 5 4 4 5 5 0 0 1 5 7 0 1 5 6 0 5 2 0 5 2 3 4 5 6 5 5 1 5 4 0 4 5 7 5 4 5 2
 5 7 0 5 5 5 5 5 2 3 2 2 4 7 6 7 5 7 7 0 4 0 5 5 2 5 2 2 6 4 2 0 4 5 7 0 2
 5 7 0 2 2 5 2 5 5 0 5 5 0 7 6 5 1 5 4 7 5 0 0 4 5 5 7 5 7 2 5 5 0 7 5 3 5
 5 5 5 0 2 1 2 4 5 5 5 2 4 0 0 4 4 4 1 7 0 2 4 4 5 0 8 5 0 2 0 0 4 2 5 5 2
 5 1 7 0 5 8 5 0 2 5 5 5 5 4 5 0 2 5 6 5 7 0 5 6 8 0 4 5 5 2 5 3 5 5 4 5 2
 5 4 5 5 7 5 0 5 5 2 2 5 1 4 0 6 2 0 5 6 0 3 2 4 0 1 2 5 7 5 8 5 5 5 0 5 5
 4 5 4 5 1 2 5 6 2 5 2 4 3 5 5 0 0 4 6 0 4 5 1 7 5 1 5 4 6 2 4 5 4 5 1 0 5
 0 4 5 5 0 5 2 5 0 5 7 5 1 4 0 7 1 5 5 4 5 5 1 5 5 5 5 7 5 5 5 4 2 5 5 5 6
 5 2 5 2 7 5 0 5 4 5 4 5 5 5 0 1 0 5 6 2 4 0 5 5 6 5 0 5 0 4 3 5 5 5 5 5 2
 2 0 7 5 7 5 2 0 7 1 5 0 

## TF-IDF预处理

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
#使用tf-idf把文本转为向量
tv = TfidfVectorizer(stop_words=stopwords, max_features=5000, lowercase = False)
tv.fit(x_train_fenci)
#模型训练
classifier.fit(tv.transform(fenci(x_train)), y_train)
#利用训练好的模型测试
classifier.score(tv.transform(fenci(x_test)), y_test)


D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.8201738334858188

In [30]:
from sklearn.metrics import classification_report
predict=classifier.predict(tv.transform(fenci(x_test)))
print("每个类别的精确率与召回率\n",classification_report(predict, y_test))

每个类别的精确率与召回率
               precision    recall  f1-score   support

           0       0.61      0.81      0.70      1044
           1       0.74      0.81      0.77      2029
           2       0.77      0.80      0.79      1531
           3       0.81      0.80      0.80      2334
           4       0.87      0.83      0.85      2855
           5       0.89      0.86      0.88      3060
           6       0.89      0.73      0.80      3999
           7       0.89      0.89      0.89      3079
           8       0.72      0.87      0.79      1929

    accuracy                           0.82     21860
   macro avg       0.80      0.82      0.81     21860
weighted avg       0.83      0.82      0.82     21860



## N-gram 模型

In [31]:
tv_2gram = TfidfVectorizer(stop_words=stopwords, max_features=5000, ngram_range=(1,2),lowercase = False)
tv_2gram.fit(x_train_fenci)
#训练模型
clf_2gram = MultinomialNB()
clf_2gram.fit(tv_2gram.transform(fenci(x_train)), y_train)
#预测
clf_2gram.score(tv_2gram.transform(fenci(x_test)), y_test)

D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.8146386093321134

In [32]:
from sklearn.metrics import classification_report
predict=clf_2gram.predict(tv_2gram.transform(fenci(x_test)))
print("每个类别的精确率与召回率\n",classification_report(predict, y_test))

每个类别的精确率与召回率
               precision    recall  f1-score   support

           0       0.60      0.83      0.70      1009
           1       0.73      0.81      0.77      1970
           2       0.78      0.79      0.78      1556
           3       0.80      0.79      0.80      2341
           4       0.87      0.82      0.84      2875
           5       0.89      0.85      0.87      3098
           6       0.89      0.72      0.79      4067
           7       0.88      0.90      0.89      3054
           8       0.71      0.87      0.78      1890

    accuracy                           0.81     21860
   macro avg       0.79      0.82      0.80     21860
weighted avg       0.82      0.81      0.82     21860



## 3-gram

In [20]:
tv_3gram = TfidfVectorizer(stop_words=stopwords, max_features=5000, ngram_range=(1,3),lowercase = False)
tv_3gram.fit(x_train_fenci)
#训练模型
clf_3gram = MultinomialNB()
clf_3gram.fit(tv_3gram.transform(fenci(x_train)), y_train)
#预测
clf_3gram.score(tv_3gram.transform(fenci(x_test)), y_test)

D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.3442622950819672

In [21]:
from sklearn.metrics import classification_report
predict=clf_3gram.predict(tv_3gram.transform(fenci(x_test)))
print("每个类别的精确率与召回率\n",classification_report(predict, y_test))

每个类别的精确率与召回率
               precision    recall  f1-score   support

           0       0.33      0.40      0.36         5
           1       0.55      0.75      0.63         8
           2       0.40      1.00      0.57         2
           3       0.00      0.00      0.00         0
           4       0.57      0.80      0.67         5
           5       1.00      0.06      0.12        31
           6       0.00      0.00      0.00         0
           7       0.50      0.38      0.43         8
           8       0.22      1.00      0.36         2

    accuracy                           0.34        61
   macro avg       0.40      0.49      0.35        61
weighted avg       0.74      0.34      0.32        61



D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
